In [ ]:
!nvidia-smi

Wed Dec 29 16:44:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git

     |████████████████████████████████| 2.5 MB 12.7 MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-l9vod10e
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-l9vod10e
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 14.1 MB 14.0 MB/s 
     |████████████████████████████████| 3.3 MB 34.3 MB/s 
     |████████████████████████████████| 51 kB 692 kB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
     |████████████████████████████████| 321 kB 52.0 MB/s 
     |████████████████████████████████| 359 kB 50.1 MB/s 
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 981 kB 43.7 MB/s 
     |████████████████████████████████| 3.0 MB 44.1 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 5.6 MB 50.6 MB/s 
     |███████

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers

In [ ]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [ ]:
# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
dicts = convert_files_to_dicts(
    dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'text': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
#}
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:
print(dicts[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/504_List_of_A_Song_of_Ice_and_Fire_video_games.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/262_Gilly__character_.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/514_Book_of_the_Stranger.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/134_Game_of_Thrones__Season_6__soundtrack_.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/347_Game_of_Thrones__season_2_.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/202_A_Feast_for_Crows.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/126_Kissed_by_Fire.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/151_Ellaria_Sa

[{'content': "''A Song of Ice and Fire'' is a series of epic fantasy novels by the American novelist and screenwriter George R. R. Martin. The novels were later on adapted to the hit HBO series ''Game of Thrones'' in 2011.", 'meta': {'name': '504_List_of_A_Song_of_Ice_and_Fire_video_games.txt'}}, {'content': "\n==Video games==\n, seven video games based on the ''A Song of Ice and Fire'' novels and ''Game of Thrones'' series have been released, with two more forthcoming. A sequel to a previously released game has been canceled. The following table showcases the correspondent title, release date, publisher, developer and the platforms on which each game was released along with any other relevant information.\n Blank cell indicates title was not released on any platform(s) by the specified manufacturers\n Cell with games console(s) indicates title was released on platform(s) by the specified manufacturers\n+ List of ''A Song of Ice and Fire'' video games\n ''A Game of Thrones: Genesis''\n

In [ ]:
retriever = ElasticsearchRetriever(document_store)

In [ ]:
from haystack.nodes import TransformersReader
reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="Nymeria travels with whom?", params={ "Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:189: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
print_answers(prediction)


Query: Nymeria travels with whom?
Answers:
[   <Answer {'answer': 'her father, Eddard', 'type': 'extractive', 'score': 0.796349287033081, 'context': "h she names Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leave", 'offsets_in_document': [{'start': 135, 'end': 153}], 'offsets_in_context': [{'start': 70, 'end': 88}], 'document_id': 'ba2a8e87ddd95e380bec55983ee7d55f', 'meta': {'name': '43_Arya_Stark.txt'}}>,
    <Answer {'answer': 'Bran, Rickon', 'type': 'extractive', 'score': 0.7302519083023071, 'context': 'elyn. Summer and Shaggydog survive the sacking of Winterfell and join Bran, Rickon, and their group on their journey to the Wall to find Jon Snow. When ', 'offsets_in_document': [{'start': 3884, 'end': 3896}], 'offsets_in_context': [{'start': 70, 'end': 82}], 'document_id': '62dbc1b045900d83a5eab9a2056eeb45', 'meta': {'name': '349_List_of_Game_of_Thrones_characters.txt'}}>,
    <Answer {'

In [ ]:
from haystack.schema import Answer, Document, Label, Span
# eval_labels = [
#        MultiLabel(labels=[Label(query="who is written in the book of life",
#        answer=Answer(answer="every person who is destined for Heaven or the World to Come",
#        offsets_in_context=[Span(374, 434)]),
#        document=Document(id='1b090aec7dbd1af6739c4c80f8995877-0',
#        content_type="text",
#        content='Book of Life - wikipedia Book of Life Jump to: navigation, search This article is about the book mentioned in Christian and Jewish religious teachings. For other uses, see The Book of Life. In Christianity and Judaism, the Book of Life (Hebrew: ספר החיים, transliterated Sefer HaChaim; Greek: βιβλίον τῆς ζωῆς Biblíon tēs Zōēs) is the book in which God records the names of every person who is destined for Heaven or the World to Come. According to the Talmud it is open on Rosh Hashanah, as is its analog for the wicked, the Book of the Dead. For this reason extra mention is made for the Book of Life during Amidah recitations during the Days of Awe, the ten days between Rosh Hashanah, the Jewish new year, and Yom Kippur, the day of atonement (the two High Holidays, particularly in the prayer Unetaneh Tokef). Contents (hide) 1 In the Hebrew Bible 2 Book of Jubilees 3 References in the New Testament 4 The eschatological or annual roll-call 5 Fundraising 6 See also 7 Notes 8 References In the Hebrew Bible(edit) In the Hebrew Bible the Book of Life - the book or muster-roll of God - records forever all people considered righteous before God'),
#        is_correct_answer=True,
#        is_correct_document=True,
#        origin="gold-label")])
#    ]